# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [1]:
#Your code here
# Import necessary libraries
import pandas as pd

# Load the dataset
data = pd.read_csv('homepage_actions.csv')

# Check the first few rows of the dataset
print(data.head())

# Check the basic statistics of the dataset
print(data.describe())

# Investigate the id column
# How many viewers also clicked?
clicks = data[data['action'] == 'click']
print(len(clicks))

# Are there any anomalies with the data; did anyone click who didn't view?
anomalies = data[(data['action'] == 'click') & (data['id'].isin(data[data['action'] == 'view']['id']) == False)]
print(anomalies)

# Is there any overlap between the control and experiment groups?
overlap = data[data['id'].duplicated(keep=False)]
print(overlap)


                    timestamp      id       group action
0  2016-09-24 17:42:27.839496  804196  experiment   view
1  2016-09-24 19:19:03.542569  434745  experiment   view
2  2016-09-24 19:36:00.944135  507599  experiment   view
3  2016-09-24 19:59:02.646620  671993     control   view
4  2016-09-24 20:26:14.466886  536734  experiment   view
                  id
count    8188.000000
mean   564699.749878
std    219085.845672
min    182988.000000
25%    373637.500000
50%    566840.500000
75%    758078.000000
max    937217.000000
1860
Empty DataFrame
Columns: [timestamp, id, group, action]
Index: []
                       timestamp      id       group action
7     2016-09-24 20:57:20.336757  349125  experiment   view
8     2016-09-24 20:58:01.948663  349125  experiment  click
11    2016-09-24 21:05:15.348935  601714  experiment   view
12    2016-09-24 21:06:27.553057  601714  experiment  click
14    2016-09-24 21:29:19.766467  487634  experiment   view
...                          ...     .

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [2]:
#Your code here

from scipy.stats import chi2_contingency

# Create a contingency table
contingency_table = pd.crosstab(data['group'], data['action'])

# Perform a chi-squared test
chi2, p, _, _ = chi2_contingency(contingency_table)

# Check the p-value
print("Chi-squared p-value:", p)


Chi-squared p-value: 0.056537191086915774


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [8]:
#Your code here
control_group = data[data['group'] == 'control']
experiment_group = data[data['group'] == 'experiment']

control_click_rate = len(control_group[control_group['action'] == 'click']) / len(control_group)
expected_experiment_clicks = len(experiment_group) * control_click_rate

expected_experiment_clicks


857.6848030018762

### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [9]:
#Your code here
import math

actual_experiment_clicks = len(experiment_group[experiment_group['action'] == 'click'])
variance = len(experiment_group) * control_click_rate * (1 - control_click_rate)
std_dev = math.sqrt(variance)

z_score = (actual_experiment_clicks - expected_experiment_clicks) / std_dev

z_score


2.71607302278784

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [10]:
#Your code here
from scipy.stats import norm

p_value = 2 * (1 - norm.cdf(abs(z_score)))  # Two-tailed test

p_value

0.0066061345518531045

### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: **Your analysis here**

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.